In [1]:
import elections as elc

districts = {
    "Østfold":            elc.districts.NorwegianFylke(1, eligible_voters=299447, area=4004),
    "Akershus":           elc.districts.NorwegianFylke(2, eligible_voters=675240, area=5669),
    "Oslo":               elc.districts.NorwegianFylke(3, eligible_voters=693494, area=454),
    "Hedmark":            elc.districts.NorwegianFylke(4, eligible_voters=197920, area=27398),
    "Oppland":            elc.districts.NorwegianFylke(5, eligible_voters=173465, area=24675),
    "Buskerud":           elc.districts.NorwegianFylke(6, eligible_voters=266478, area=14920),
    "Vestfold":           elc.districts.NorwegianFylke(7, eligible_voters=246041, area=2168),
    "Telemark":           elc.districts.NorwegianFylke(8, eligible_voters=173355, area=15298),
    "Aust-Agder":         elc.districts.NorwegianFylke(9, eligible_voters=118273, area=9155),
    "Vest-Agder":         elc.districts.NorwegianFylke(10, eligible_voters=188958, area=7278),
    "Rogaland":           elc.districts.NorwegianFylke(11, eligible_voters=479892, area=9377),
    "Hordaland":          elc.districts.NorwegianFylke(12, eligible_voters=528127, area=15438),
    "Sogn og Fjordane":   elc.districts.NorwegianFylke(14, eligible_voters=108404, area=18433),
    "Møre og Romsdal":    elc.districts.NorwegianFylke(15, eligible_voters=265238, area=14356),
    "Sør-Trøndelag":      elc.districts.NorwegianFylke(16, eligible_voters=334514, area=20257),
    "Nord-Trøndelag":     elc.districts.NorwegianFylke(17, eligible_voters=134188, area=21944),
    "Nordland":           elc.districts.NorwegianFylke(18, eligible_voters=241235, area=38155),
    "Troms Romsa":        elc.districts.NorwegianFylke(19, eligible_voters=167839, area=26198),
    "Finnmark Finnmárku": elc.districts.NorwegianFylke(20, eligible_voters=75472, area=48631),
}

In [2]:
mandates = elc.distributions.StLague(169, initial_divisor=1)

for name, district in districts.items():
    district.name = name
    mandates.add_score(name, 1.8*district.area + district.eligible_voters)

mandates.calculate()

for name, seats in mandates.result.items():
    district = districts[name]
    district.distribution = elc.distributions.StLague(seats - 1, initial_divisor=1.4) # subtract one seat, because it is used for leveling seats
    district.available_leveling_seats = 1    

In [3]:
import pandas as pd
results_2021 = pd.read_csv("2021-09-21_partydist_final.csv", delimiter=";")
results_2021.head()

,Fylkenummer,Fylkenavn,Kommunenummer,Kommunenavn,Stemmekretsnummer,Stemmekretsnavn,Partikode,Partinavn,Oppslutning prosentvis,Antall stemmeberettigede,Antall forhåndsstemmer,Antall valgtingstemmer,Antall stemmer totalt,Endring % siste tilsvarende valg,Endring % siste ekvivalente valg,Antall mandater,Antall utjevningsmandater,Unnamed: 17
0,1,Østfold,NaN,NaN,NaN,NaN,A,Arbeiderpartiet,"30,47944",223945,24817,24528,49345,"-1,59066","3,00272",3,0,NaN
1,1,Østfold,NaN,NaN,NaN,NaN,SV,SV - Sosialistisk Venstreparti,"6,07798",223945,5525,4315,9840,"1,71448","1,87632",1,1,NaN
2,1,Østfold,NaN,NaN,NaN,NaN,RØDT,Rødt,"4,58195",223945,4258,3160,7418,"2,44022","0,40159",0,0,NaN
3,1,Østfold,NaN,NaN,NaN,NaN,SP,Senterpartiet,"14,11338",223945,10082,12767,22849,"5,4521","-3,17734",2,0,NaN
4,1,Østfold,NaN,NaN,NaN,NaN,KRF,Kristelig Folkeparti,"3,34412",223945,2576,2838,5414,"-0,86063","-0,23466",0,0,NaN


In [4]:
parties = {}
for partikode in results_2021.Partikode.unique():
    partinavn = results_2021[results_2021.Partikode == partikode].Partinavn.unique() # Pandas magic to get a list of matching partinavn for each partikode
    if len(partinavn) != 1:
        print(f"Party ID {partikode} does not match a single party name")
    total_party_votes = results_2021[results_2021.Partikode == partikode]["Antall stemmer totalt"].sum()
    parties[partikode] = elc.Party(partinavn[0], total_votes=total_party_votes)

In [5]:
""" Calculate the direct seat distribution """
results_district_names = results_2021.Fylkenavn.unique()
for district_name in results_district_names:
    district = districts[district_name]
    results_this_district = results_2021[results_2021.Fylkenavn == district_name]
    
    for party_id, party in parties.items():
        results_this_party = results_this_district[results_this_district.Partikode == party_id]
        party_votes_this_district = results_this_party["Antall stemmer totalt"].sum()
        district.distribution.add_score(party, party_votes_this_district)

    for party, seats in district.result.items():
        party.seats_awarded += seats

In [6]:
""" Calculate leveling seats per party """
national_distribution = elc.distributions.StLague(169, initial_divisor=1.4)
for party_id, party in parties.items():
    if party_id == "BLANKE":
        continue
        
    national_distribution.add_score(party, party.total_votes)

true_distribution = national_distribution.true_distribution # true, fractional distribution
percentage_share = national_distribution.score_share # true, precentage share of votes
    
for party, percent_votes in percentage_share.items():
    if percent_votes < 4:
        national_distribution.remove_candidate(party)
        national_distribution.num_seats -= party.seats_awarded

leveling_results = national_distribution.result
overrepresented_parties = True
while overrepresented_parties:
    overrepresented_parties = False
        
    for party, leveling_seats in leveling_results.items():
        if party.seats_awarded >= leveling_seats:
            overrepresented_parties = True
            national_distribution.remove_candidate(party)
            national_distribution.num_seats -= party.seats_awarded
            
    leveling_results = national_distribution.result

In [7]:
for party_id, party in parties.items():
    tot_seats = party.seats_awarded
    
    before_leveling = tot_seats 
    if party in leveling_results:
        tot_seats = leveling_results[party]
    leveling_seats = tot_seats - before_leveling
    
    if tot_seats == 0:
        continue
        
    print(f"{party.name:>40s} {tot_seats:>3d} {leveling_seats:>2d} {tot_seats - true_distribution[party]} {percentage_share[party]}")
    


                         Arbeiderpartiet  48  0 3.6349565224967435 26.251505016274113
          SV - Sosialistisk Venstreparti  13  5 0.0843727286527276 7.642383000797202
                                    Rødt   8  4 0.018818190683090208 4.722592786578053
                           Senterpartiet  28  0 5.179577218183713 13.503208746636854
                    Kristelig Folkeparti   3  0 -3.4188792458381467 3.7981533999042285
                  Miljøpartiet De Grønne   3  0 -3.66256605232849 3.942346776525734
                                 Venstre   8  5 0.21691636616606136 4.605374931262685
                                   Høyre  36  1 1.6065776038834017 20.351137512495026
                      Fremskrittspartiet  21  4 1.3785399507470544 11.6103313900905
                            Pasientfokus   1  0 0.7196723931844753 0.165874323559482
